In [1]:
import pandas as pd
import numpy as np
import datasets
import os
import sys
import ast
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
import utils

human_data = pd.read_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/all_human_annotations_processed.csv')
aspects = [ 'actionability', 'grounding_specificity','verifiability', 'helpfulness', 'addressed_to_author','valid_point','professional_tone']

accepted_annotators = ['boda', '6740484e188a64793529ee77', '6686ebe474531e4a1975636f']

In [2]:
human_data.head(3)

,review_point,paper_id,venue,focused_review,actionability,actionability_label,actionability_label_type,batch,grounding_specificity,grounding_specificity_label,...,helpfulness_label_type,professional_tone,professional_tone_label,professional_tone_label_type,valid_point,valid_point_label,valid_point_label_type,addressed_to_author,addressed_to_author_label,addressed_to_author_label_type
0,- There were too many missing details (for exa...,ARR_2022_60_review,ARR_2022,- Underdefined and conflation of concepts - Se...,"{'annotators': ['boda'], 'labels': ['4']}",NaN,NaN,1,"{'annotators': ['boda'], 'labels': ['3']}",NaN,...,NaN,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,"{'annotators': ['boda'], 'labels': ['0']}",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
1,"- Section 4.3, model parameters and training. ...",ACL_2017_333_review,ACL_2017,There are some few details on the implementati...,"{'annotators': ['boda'], 'labels': ['5']}",NaN,NaN,1,"{'annotators': ['boda'], 'labels': ['5']}",NaN,...,NaN,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,"{'annotators': ['boda'], 'labels': ['0']}",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN
2,- The differences in results in Table 2 are ve...,ARR_2022_269_review,ARR_2022,- It is not clear for me about the novelty of ...,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,1,"{'annotators': ['boda'], 'labels': ['5']}",NaN,...,NaN,"{'annotators': ['boda'], 'labels': ['1']}",NaN,NaN,"{'annotators': ['boda'], 'labels': ['0']}",NaN,NaN,"{'annotators': [], 'labels': []}",NaN,NaN


In [3]:
from collections import Counter

def most_frequent_value(arr):
    count = Counter(arr)
    most_common_value, frequency = count.most_common(1)[0]
    return most_common_value, frequency

In [4]:
processed_data = {}
for aspect in aspects:
    rows = []
    include_columns = ['review_point','paper_id','venue','focused_review','batch', f'{aspect}',f'{aspect}_label', f'{aspect}_label_type']
    aspect_data = human_data[include_columns]

    ### add ID column
    aspect_data['id'] = aspect_data.index

    for index, row in aspect_data.iterrows():
        temp_row = row.copy()
        labels_dict = ast.literal_eval(row[f'{aspect}'])
        new_labels = []
        new_annotators = []

        # if 'boda' not in labels_dict['annotators']:
        #     print(row)
        
        for annotator, label in zip(labels_dict['annotators'], labels_dict['labels']):
            if utils.annotators_unique_id_batch_id_map_inv[annotator] in accepted_annotators:
                new_labels.append(label)
                new_annotators.append(utils.annotators_unique_id_batch_id_map_inv[annotator])

        temp_row[f'{aspect}'] = {'annotators': new_annotators, 'labels': new_labels}

        if len(new_labels) < 3:
            continue

        value, freq = most_frequent_value(new_labels)
        if freq >= 3:
            temp_row[f'{aspect}_label'] = value
            temp_row[f'{aspect}_label_type'] = 'gold'
        elif freq == 2:
            temp_row[f'{aspect}_label'] = value
            temp_row[f'{aspect}_label_type'] = 'silver'
        else:
            temp_row[f'{aspect}_label'] = value
            temp_row[f'{aspect}_label_type'] = 'hard'
        
        rows.append(temp_row)
        


    processed_data[aspect] = pd.DataFrame(rows)

/tmp/ipykernel_1653529/1811724354.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aspect_data['id'] = aspect_data.index
/tmp/ipykernel_1653529/1811724354.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aspect_data['id'] = aspect_data.index
/tmp/ipykernel_1653529/1811724354.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [5]:
hf_data = {}
for aspect in aspects:
    ds = {}

    ds['gold'] = processed_data[aspect][processed_data[aspect][f'{aspect}_label_type'] == 'gold']
    ds['silver'] = processed_data[aspect][processed_data[aspect][f'{aspect}_label_type'] == 'silver']
    ds['hard'] = processed_data[aspect][processed_data[aspect][f'{aspect}_label_type'] == 'hard']
    ds['all'] = processed_data[aspect]

    hf_data[aspect] = ds


In [9]:
for aspect in aspects:
    for split in ds.keys():
        print(aspect, split, hf_data[aspect][split].shape)
        if hf_data[aspect][split].shape[0] == 0:
            continue
        hd_ds = datasets.Dataset.from_pandas(hf_data[aspect][split])

        if  '__index_level_0__' in hd_ds.column_names:
            hd_ds = hd_ds.remove_columns(['__index_level_0__'])

        hd_ds.push_to_hub(
            f"boda/review_evaluation_human_annotation",
            split=split,
            config_name=aspect, private=True)

    ## wait for 3 seconds
    print('Waiting for 3 seconds...')
    os.system("sleep 60")

actionability gold (444, 9)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/6.62k [00:00<?, ?B/s]

actionability silver (764, 9)


...

professional_tone hard (0, 9)
professional_tone all (1422, 9)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

Waiting for 3 seconds...


In [15]:
# for key in hf_data.keys():
#     dataset_dict = datasets.DatasetDict()
#     for sub_key in hf_data[key].keys():
#         aspect_hf_ds = datasets.Dataset.from_pandas(hf_data[key][sub_key])
#         dataset_dict[sub_key] = aspect_hf_ds

#     aspect_hf_ds.push_to_hub(f"boda/review_evaluation_human_annotation", config_name=key, private=True)